In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

csv_file = "/Users/Shrujai/Downloads/Keywords of events(day-wise) - March-May 2020 (1).csv"
event_df = pd.read_csv(csv_file)
pd.set_option("display.max_colwidth",1000)

event_df.head(60)

FileNotFoundError: ignored

In [ ]:
#helper functions

import csv
import re

def calculate_emotion(date, emotion, event):
    emotion = emotion.lower()
    csv_file_path = f"/Users/shrujai/Desktop/Tweets data/{date}_labelled.csv"
    emotion_numbers = []

    data_list = []

    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            data_list.append(row)

    for data in data_list:
        if re.findall(event, data["content"], flags=re.IGNORECASE):
            emotion_numbers.append(float(data[emotion]))  # Changed to append float value

    return emotion_numbers

In [ ]:
#helper functions

import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")

def getKeywords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            if(token.text== "covid" or token.text== "covid-19"):
                continue

            result.append(token.text)
    return result

import math
e = math.e

def sigmoid(PEI):
    PEI = -1*PEI
    return 1/(1+ e**PEI)


In [ ]:
def TSA(CP, emo, event, event_df, CPW, date):
    if(event==""):
        print("Please enter a valid event")
        return -1
    keywords = getKeywords(event);
    csv_file_path = f"/Users/shrujai/Desktop/Tweets data/{date}_labelled.csv"

    emotion_list = []

    for keyword in keywords:
        emotion_numbers = calculate_emotion(date, emo, keyword)
        emotion_list.extend(emotion_numbers)

    threshold = 0.16
    countForThreshold=0
    for score in emotion_list:
        if(score>=threshold):
            countForThreshold=countForThreshold+1

    PEI = countForThreshold/len(emotion_list)
    PEI_dash = (len(emotion_list)-countForThreshold)/(len(emotion_list))

    direction = ""
    flag = -1

    if(PEI<PEI_dash):
        direction = "decreasing"
        flag = 0
    else:
        direction = "increasing"
        flag = 1;

    if(flag==0):
        return sigmoid(PEI_dash), direction, event, emo
    else:
        return sigmoid(PEI), direction, event, emo

In [ ]:
cp = 0
event = "Dr VK Paul from NITI Aayog presented a chart predicting that India would have zero new cases by May 16. On May 16, India saw 4,987 cases, the highest single-day spike."
prob, direction, event, emo = TSA(cp,"joy", event ,event_df, 14,"2020-04-24")


print(prob)
print(direction)
print(event)
print(emo)

0.7020768086452889
increasing
Dr VK Paul from NITI Aayog presented a chart predicting that India would have zero new cases by May 16. On May 16, India saw 4,987 cases, the highest single-day spike.
joy
